In [11]:
pip install pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 409.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 1.4 MB/s eta 0:00:001.3 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.5 MB/s eta 0:00:00m eta 0:00:010:01:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [58]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

DATABASE_URL = "mysql+pymysql://root:@localhost:3306/pro_boost"

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()


In [59]:
from sqlalchemy import Column, Integer, String, Boolean, DateTime, Enum, Text, DECIMAL, ForeignKey
from sqlalchemy.sql import func
import enum
from datetime import datetime

class UserRole(enum.Enum):
    buyer = "buyer"
    pro = "pro"
    admin = "admin"

class User(Base):
    __tablename__ = "users"
    user_id = Column(Integer, primary_key=True, autoincrement=True)
    email = Column(String(255), unique=True, nullable=False)
    password_hash = Column(Text, nullable=False)
    role = Column(Enum('buyer', 'pro', 'admin'), nullable=False)
    name = Column(String(100), nullable=True)
    avatar_url = Column(Text, nullable=True)
    created_at = Column(DateTime, server_default=func.now())
    is_verified = Column(Boolean, default=False)
    status = Column(String(20), nullable=False, default='inactive')

class BlogPost(Base):
    __tablename__ = "blog_posts"
    post_id = Column(Integer, primary_key=True, index=True)
    author_id = Column(Integer, ForeignKey('users.user_id'), nullable=False)
    title = Column(String(255), nullable=True)
    slug = Column(String(255), unique=True, nullable=True)
    content = Column(Text, nullable=True)
    cover_image_url = Column(Text, nullable=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, nullable=True)

class BlogComment(Base):
    __tablename__ = "blog_comments"
    comment_id = Column(Integer, primary_key=True, index=True)
    post_id = Column(Integer, ForeignKey('blog_posts.post_id'), nullable=False)
    user_id = Column(Integer, ForeignKey('users.user_id'), nullable=True)
    content = Column(Text, nullable=True)
    created_at = Column(DateTime, default=datetime.utcnow)

class Game(Base):
    __tablename__ = "games"
    game_id = Column(Integer, primary_key=True, index=True)
    name = Column(String(100), nullable=False)
    icon_url = Column(Text, nullable=True)

class Service(Base):
    __tablename__ = "services"
    service_id = Column(Integer, primary_key=True, index=True)
    game_id = Column(Integer, ForeignKey('games.game_id'), nullable=False)
    name = Column(String(100), nullable=False)
    description = Column(Text, nullable=True)
    price_per_unit = Column(DECIMAL(10,2), nullable=True)
    icon_url = Column(Text, nullable=True)
    sale_price = Column(DECIMAL(10,2), nullable=True)
    category = Column(String(100), nullable=True)

class Order(Base):
    __tablename__ = "orders"
    order_id = Column(Integer, primary_key=True, index=True)
    buyer_id = Column(Integer, ForeignKey('users.user_id'), nullable=False)
    service_id = Column(Integer, ForeignKey('services.service_id'), nullable=False)
    game_id = Column(Integer, ForeignKey('games.game_id'), nullable=False)
    status = Column(Enum('pending','in_progress','completed','cancelled', name='order_status'), default='pending')
    details = Column(Text, nullable=True)
    created_at = Column(DateTime, default=datetime.utcnow)

class OrderAssignment(Base):
    __tablename__ = "order_assignments"
    assignment_id = Column(Integer, primary_key=True, index=True)
    order_id = Column(Integer, ForeignKey('orders.order_id'), nullable=False)
    pro_id = Column(Integer, ForeignKey('users.user_id'), nullable=False)
    assigned_at = Column(DateTime, default=datetime.utcnow)
    completed_at = Column(DateTime, nullable=True)
    remarks = Column(Text, nullable=True)

class ProApplication(Base):
    __tablename__ = "pro_applications"
    application_id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer, ForeignKey('users.user_id'), nullable=False)
    experience = Column(Text, nullable=True)
    status = Column(Enum('pending','approved','rejected', name='application_status'), default='pending')
    submitted_at = Column(DateTime, default=datetime.utcnow)

class ChatMessage(Base):
    __tablename__ = "chat_messages"
    message_id = Column(Integer, primary_key=True, index=True)
    order_id = Column(Integer, ForeignKey('orders.order_id'), nullable=False)
    sender_id = Column(Integer, ForeignKey('users.user_id'), nullable=False)
    receiver_id = Column(Integer, ForeignKey('users.user_id'), nullable=False)
    message = Column(Text, nullable=False)
    sent_at = Column(DateTime, default=datetime.utcnow)


In [60]:
from pydantic import BaseModel
from sqlalchemy.orm import Session

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

class GameCreate(BaseModel):
    name: str
    icon_url: str = None

In [61]:
db = next(get_db())

In [28]:
# create pandas dataframe get all game Name from folder gamesTwo
import os
import pandas as pd
gamesTwo_files = set(os.listdir("gamesTwo"))
all_games = []
for file_name in gamesTwo_files:
    df = pd.read_csv(f'./gamesTwo/{file_name}')
    all_games.append(df.iloc[0,0])



In [56]:
import re
import requests
gamesTwo_images = list(os.listdir("game_services"))
gamesTwo_images.sort()
errored = []
all_images = []
for game in gamesTwo_images:
    match = re.search(r"skycoach_results_(.*?)_services\.csv(?:\.csv)?", game)

    if match:
        game_name = match.group(1).replace('_', ' ')
        print(game_name)  # Output: 8 Ball Pool
    try:
        df = pd.read_csv(f'./game_services/{game}')
        print(df.iloc[0,6])
        all_images.append(df.iloc[0,6])
    except Exception as e:
        print(f"Error reading {game}: {e}")
        game_instance = Game(name=game_name, icon_url="")
        # db.add(game_instance)
        # db.commit()
        # db.refresh(game_instance)
        # print(f"Added game: {game_name} without icon URL due to error: {e}")
        errored.append(game)
        continue
    if df.empty:
        print(f"Skipping empty dataframe for {game_name}")
        continue
    # break

    # game_instance = Game(name=game_name, icon_url=df.iloc[0,6])
    # db.add(game_instance)
    # db.commit()
    # db.refresh(game_instance)
    # print(f"Added game: {game_name} with icon URL: {df.iloc[0,6]}")

for game_image in all_images:
    try:
        response = requests.get(game_image)
        # save in lcoal
        with open(f'./game_images/{game_image.split("/")[-1]}', 'wb') as f:
            f.write(response.content)
        # Check if the image exists
        print(f"Checking image: {game_image}")
        response = requests.head(game_image)
        if response.status_code == 200:
            
            print(f"Image {game_image} exists.")
        else:
            print(f"Image {game_image} does not exist. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Error checking image {game_image}: {e}")

8 Ball Pool
Error reading skycoach_results_8_Ball_Pool_services.csv.csv: No columns to parse from file
ARC Raiders
https://skycoach.gg/storage/uploads/products/arc-raiders-dam-battlegrounds-raid1749064142_picture_item.png
Albion Online
https://skycoach.gg/storage/uploads/products/albion-online-tomes-of-insights1736542026_picture_item.png
Apex Legends
https://skycoach.gg/storage/uploads/products/ranked-rumble-boost1722685907_picture_item.png
Arena Breakout- Infinite
https://skycoach.gg/storage/uploads/products/abi-raids-boost1723469655_picture_item.png
Ashes of Creation
https://skycoach.gg/storage/uploads/products/aoc-rogue-build1744896612_picture_item.png
Black Desert Online
https://skycoach.gg/storage/uploads/products/bdo-season-pass-boost-full-completion1698783753_picture_item.png
Blade & Soul NEO
https://skycoach.gg/storage/uploads/products/blade-soul-neo-quests-boost1742920941_picture_item.png
Brawl Stars
https://skycoach.gg/storage/uploads/products/championship-challenge-boost1739

In [62]:
import re
gamesTwo_images = list(os.listdir("game_services"))
gamesTwo_images.sort()
errored = []
for game in gamesTwo_images:
    match = re.search(r"skycoach_results_(.*?)_services\.csv(?:\.csv)?", game)

    if match:
        game_name = match.group(1).replace('_', ' ')
        print(game_name)  # Output: 8 Ball Pool
    try:
        df = pd.read_csv(f'./game_services/{game}')
        print(df.iloc[0,6])
    except Exception as e:
        print(f"Error reading {game}: {e}")
        game_instance = Game(name=game_name, icon_url="")
        db.add(game_instance)
        db.commit()
        db.refresh(game_instance)
        print(f"Added game: {game_name} without icon URL due to error: {e}")
        errored.append(game)
        continue
    if df.empty:
        print(f"Skipping empty dataframe for {game_name}")
        continue
    # break

    game_instance = Game(name=game_name, icon_url=df.iloc[0,6])
    db.add(game_instance)
    db.commit()
    db.refresh(game_instance)
    print(f"Added game: {game_name} with icon URL: {df.iloc[0,6]}")

8 Ball Pool
Error reading skycoach_results_8_Ball_Pool_services.csv.csv: No columns to parse from file
Added game: 8 Ball Pool without icon URL due to error: No columns to parse from file
ARC Raiders
https://skycoach.gg/storage/uploads/products/arc-raiders-dam-battlegrounds-raid1749064142_picture_item.png
Added game: ARC Raiders with icon URL: https://skycoach.gg/storage/uploads/products/arc-raiders-dam-battlegrounds-raid1749064142_picture_item.png
Albion Online
https://skycoach.gg/storage/uploads/products/albion-online-tomes-of-insights1736542026_picture_item.png
Added game: Albion Online with icon URL: https://skycoach.gg/storage/uploads/products/albion-online-tomes-of-insights1736542026_picture_item.png
Apex Legends
https://skycoach.gg/storage/uploads/products/ranked-rumble-boost1722685907_picture_item.png
Added game: Apex Legends with icon URL: https://skycoach.gg/storage/uploads/products/ranked-rumble-boost1722685907_picture_item.png
Arena Breakout- Infinite
https://skycoach.gg/st

In [44]:
all_games.sort()
all_games

['8 Ball Pool',
 'ARC Raiders',
 'Albion Online',
 'Apex Legends',
 'Arena Breakout- Infinite',
 'Ashes of Creation',
 'Black Desert Online',
 'Blade & Soul NEO',
 'Brawl Stars',
 'Call of Duty',
 'Call of Duty Mobile',
 'Clash Royale',
 'Clash of Clans',
 'Corepunk',
 'Counter-Strike 2',
 'Dark and Darker',
 'Dead by Daylight',
 'Deadlock',
 'Delta Force Hawk Ops',
 'Destiny 2',
 'Diablo 4',
 'Diablo Immortal',
 'Division 2',
 'Dota 2',
 'Dragon Ball Legends',
 'Dune- Awakening',
 'EVE Online',
 'Elden Ring',
 'Elden Ring Nightreign',
 'Escape from Tarkov',
 'FC 25',
 'Fallout 76',
 'Final Fantasy XIV',
 'Fortnite',
 'Forza Horizon 5',
 'FragPunk',
 'Genshin Impact',
 'GoT Kingsroad',
 'Guild Wars 2',
 'Hay Day',
 'Hearthstone',
 'Helldivers 2',
 'Honkai Star Rail',
 'Hunt Showdown',
 'Last Epoch',
 'League of Legends',
 'Lost Ark',
 'MapleStory',
 'Marvel Rivals',
 'Minecraft',
 'MoCo',
 'Mobile Legends',
 'Monopoly Go',
 'Monster Hunter Wilds',
 'NBA',
 'New World',
 'Old School Run

In [94]:
def extract_numeric(value):
    import re
    if not isinstance(value, str):
        return None
    match = re.findall(r'[\d,\.]+', value)
    if not match:
        return None
    return float(match[0].replace(',', '.'))

In [ ]:
# save the services of games by getting game id
db = next(get_db())
all_games = list(os.listdir("game_services"))
errored_games
for game in all_games:
    try:
        game_name = game.replace('skycoach_results_', '').replace('_services.csv.csv', '').replace('_', ' ')
        print(game_name)
        if game_name == 'Last Epoch':
            continue  # Skip Last Epoch as per the original code logic
        game_instance = db.query(Game).filter(Game.name == game_name).first()
        print('game instance')
        if not game_instance:
            print(f"Game {game} not found in database.")
            continue
        df = pd.read_csv(f'./game_services/{game}')
        for index, row in df.iterrows():
            service_instance = Service(
                game_id=game_instance.game_id,
                name=row['Name'],
                description=row['Description'],
                price_per_unit=extract_numeric(row['Price']),
                icon_url=row['GameCharImage'],
                sale_price=row.get('sale_price', 0.0),
                category=row.get('Category', "")
            )
            db.add(service_instance)
        db.commit()
        db.refresh(game_instance)
        print(f"Added services for game: {game}")
    except Exception as e:
        print(f"Error processing game {game}: {e}")
    # print(game)
    # break

Last Epoch
Old School RuneScape
game instance
Added services for game: skycoach_results_Old_School_RuneScape_services.csv.csv
WoW Mists of Pandaria Classic
game instance
Error processing game skycoach_results_WoW_Mists_of_Pandaria_Classic_services.csv.csv: could not convert string to float: '2.699.99'
Zenless Zone Zero
game instance
Added services for game: skycoach_results_Zenless_Zone_Zero_services.csv.csv
Lost Ark
game instance
Added services for game: skycoach_results_Lost_Ark_services.csv.csv
World of Warcraft
game instance
Error processing game skycoach_results_World_of_Warcraft_services.csv.csv: could not convert string to float: '1.799.99'
Tarisland
game instance
Error processing game skycoach_results_Tarisland_services.csv.csv: could not convert string to float: '1.099.99'
Rematch
game instance
Added services for game: skycoach_results_Rematch_services.csv.csv
Tower of Fantasy
game instance
Added services for game: skycoach_results_Tower_of_Fantasy_services.csv.csv
Warcraft Ru